In [625]:
import numpy as np
import pandas as pd

In [626]:
train = pd.read_csv('train.csv', index_col=0)

In [627]:
train1 = train.copy()

In [628]:
#train1.drop('MSSubClass', axis=1, inaplce=True)

In [629]:
#ob with a basement, but shows "no basement" in the BsmtExposure col NEED TO IMPUTE - same as bsmtcond
train1[train1['BsmtExposure'].isnull() & train1['BsmtQual'].notnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
949,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2006,WD,Normal,192500


In [630]:
import random
random.seed(0)
#obs949 is randomly assigned a a non-"No" categery for BsmtExposure
train1['BsmtExposure'][949]=train1[train1['BsmtExposure']!='No'].sample()['BsmtExposure']

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [631]:
#ob with a BsmtFinSF2 but shows "no basement" in the BsmtFinType2 col NEED TO IMPUTE  - random
train1[train1['BsmtFinType2'].isnull() & train1['BsmtFinSF2']!=0]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
333,20,RL,85.0,10655,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,10,2009,WD,Normal,284000


In [632]:
random.seed(0)
#obs333 is randomly assigned a non-"No" categery for BsmtFinType2
train1['BsmtFinType2'][333]=train1[train1['BsmtFinType2']!='nan'].sample()['BsmtFinType2'].values[0]

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [633]:
#ob with Na in Electrical 
train1[train1['Electrical'].isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1380,80,RL,73.0,9735,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,167500


In [634]:
random.seed(0)
#obs1380 is randomly assigned a random
train1['Electrical'][1380] = train1['Electrical'].sample().values[0]

/Users/zhuwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [635]:
#for BsmtExposure - replace No with NoExposure; replace NaN with NoBsmt
train1['BsmtExposure']=train1['BsmtExposure'].str.replace('No','NoExposure')

In [636]:
train1['BsmtExposure'].replace(np.NaN,'NoBsmt',inplace=True)

In [637]:
#GarageYrBlt - change No to numerical zero 
train1['GarageYrBlt'].replace(np.NaN,0,inplace=True)

In [638]:
#cols for which NA will be replaced with "No" because NA is a true category thus need to change 
#to "No" so Pandas does not pick it up as np.NaN
FixNA=['Alley', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2','FireplaceQu', 'GarageType','GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

In [639]:
#change np.NaN to "No"
for v in FixNA:
    train1[v].replace(np.NaN,'No',inplace=True)

In [640]:
#spread MiscFeature, replace NaN with zero, and drop No. 
misc=train1[['MiscFeature','MiscVal']]
misc=pd.pivot_table(misc,index='Id',columns='MiscFeature',values='MiscVal')
misc.replace(np.NaN,0,inplace=True)
misc=misc.drop(['No','Othr'],axis=1).apply(pd.to_numeric, errors='ignore', downcast='integer')

In [641]:
#combined misc with original DF and drop MiscFeature, MiscVal
train1 = pd.concat([train1.drop(['MiscFeature','MiscVal'],axis=1),misc],axis=1)

In [642]:
train1['Electrical'].value_counts()

SBrkr    1335
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [643]:
#list of categorical variables 
cat_variables=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour','Utilities',
               'LotConfig', 'LandSlope','Neighborhood', 'Condition1', 'Condition2',
               'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl','Exterior1st', 
               'Exterior2nd', 'MasVnrType', 'Foundation','BsmtExposure','BsmtFinType2',
               'Heating','CentralAir','Electrical', 'GarageType','SaleType', 'SaleCondition']


In [644]:
#dummify categorical nominal and binary variables
for v in cat_variables:
    temp = pd.get_dummies(train1[v],prefix=v, prefix_sep='_')
    temp = temp.drop(temp.iloc[:,0:1],axis=1)
    train1 = pd.concat([train1.drop(v,axis=1),temp],axis=1)


In [686]:
#list for Condition1 and Condition2
Condition1 = list(train1.filter(regex='Condition1').columns)
Condition1.remove('Condition1_RRNe')
Condition2 = list(train1.filter(regex='Condition2').columns)

In [695]:
#combine all Condition1 and Condition2 columns and drop Condition2
for i,j in zip(Condition1,Condition2):
    train1[i] = train1[i] | train1[j]
    train1.drop(j,axis=1,inplace=True)

In [701]:
train1.shape

(1460, 205)